In [ ]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

Load the dataset

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
subm = pd.read_csv('../input/sample_submission.csv')

# Looking at the data

The training data contains a row per comment, with an id, the text of the comment, and 6 different labels that we'll try to predict.

In [ ]:
train.head()

look at some beautiful datas

In [ ]:
for i in range(5):
    print(train["comment_text"][i])

In [ ]:
for i in range(5):
    print(test["comment_text"][i])

I dont know about data science but by looking at the data my patience got improved by 34.87%

As length is every thing of a text lets analyze its length 

In [ ]:
lens = train.comment_text.str.len()
lens.mean(), lens.std(), lens.max(),lens.min(),lens.median()



As median is way before the mean we can surely say the lengths of the text formed a positively skewed distribution

In [ ]:
#see the length of lines
fig,axis1=plt.subplots(1,1,figsize=(8,4))
sns.countplot(lens,ax=axis1)

lets find out missing columns stats

In [ ]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['none'] = 1-train[label_cols].max(axis=1)
train.describe(),train.info()

In [ ]:
len(train),len(test)

thanks to god we are not suffering from bias problem 

see the stat for categorical variable

In [ ]:
train.describe(include=["O"])

In [ ]:
test.describe(include=["O"])

none of the statesment are repetative it seems the quality of data sounds perfect to apply some ML

In [ ]:
train['comment_text'].fillna("unknown", inplace=True)
test['comment_text'].fillna("unknown", inplace=True)

[http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html](http://) go to the link to learn about CountVectorizer if you are unfamilier

In [ ]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [ ]:
n = train.shape[0]
vec = CountVectorizer(ngram_range=(1,3), tokenizer=tokenize, max_features=20000)
trn_term_doc = vec.fit_transform(train['comment_text'])
test_term_doc = vec.transform(test['comment_text'])

In [ ]:
trn_term_doc, test_term_doc

In [ ]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [ ]:
x=trn_term_doc.sign()
test_x = test_term_doc.sign()

In [ ]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=0.1, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [ ]:
preds = np.zeros((len(test), len(label_cols)))

for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

In [ ]:
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
submission.to_csv('submission.csv', index=False)